In [11]:
import glob
import json

from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.document_loaders import JSONLoader

files = glob.glob("../data/*_cn.json")

def _metadata_fn(record: dict, metadata: dict) -> dict:
    for k, v in record.items():
        if k != "review":
            metadata[k] = v
    return metadata


loader = JSONLoader(
    file_path=files[0],
    jq_schema=".reviews[]",
    content_key="review",
    metadata_func=_metadata_fn
)

data = loader.load()
print(f"Loaded {len(data)} records from {files[0]}")
print(json.dumps(data[0].metadata, indent=2))
print(data[0].page_content)


Loaded 20 records from ../data/steam_monster_hunter_world_reviews_cn.json
{
  "source": "/home/purui/test/offline_langchain_rag/data/steam_monster_hunter_world_reviews_cn.json",
  "seq_num": 1,
  "recommendationid": "191775636",
  "author": {
    "steamid": "76561199029436405",
    "num_games_owned": 347,
    "num_reviews": 7,
    "playtime_forever": 40290,
    "playtime_last_two_weeks": 0,
    "playtime_at_review": 40243,
    "last_played": 1743826482
  },
  "language": "schinese",
  "timestamp_created": 1743566178,
  "timestamp_updated": 1743566472,
  "voted_up": true,
  "votes_up": 7,
  "votes_funny": 2,
  "weighted_vote_score": "0.599085867404937744",
  "comment_count": 0,
  "steam_purchase": true,
  "received_for_free": false,
  "written_during_early_access": false,
  "primarily_steam_deck": false
}
[h1] 写在最前面：怪物猎人不是适合所有人的游戏 [/h1]
        转眼已经是快700小时的xdx（小东西）了，也是时候写个测评给人吐槽了（
        黑框嘛，一般是带有较强烈的个人情感，可以略过，尤其是大段黑框
        游戏肯定是给推荐的，[strike]不喜欢我也玩不了这么久， [/strike]毕竟是卡表的看家ip之一，而且更新到完结

In [15]:
_MODEL_NAME = "deepseek-r1:7b"

In [ ]:
from langchain_ollama import OllamaEmbeddings

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OllamaEmbeddings(model=_MODEL_NAME),
).from_loaders([loader])

/home/purui/test/offline_langchain_rag/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [19]:
from langchain_ollama.llms import OllamaLLM
model = OllamaLLM(model=_MODEL_NAME)



In [ ]:
import langchain
langchain.debug = True

response = index.query("这游戏适合新手吗？", llm=model, retriever_kwargs={"k": 5})
print(response)

langchain.debug = False

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "这游戏适合新手吗？"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "这游戏适合新手吗？",
  "context": "怎么说呢，断断续续玩怪猎200多个小时了，每次打开怪猎都有新的体验，换一把武器就是一个新的游戏，打不过了就摇人，天尊各个心态好又友善，基本不会责怪新手，游戏的氛围是真真的没谁了，不过对于现在荒野都开了还在留守冰原的我来说，再买一个新的怪猎来玩实在有些有心无力了。\n\n生态嘛...有人吹有人骂。在游戏允许的情况下加能被玩家看到的细节，增加代入感以及游戏性肯定是好事，不过....，台阶，不，tgy（藤冈要），身比碳钢硬，柠檬碎龙[spoiler] sbtgy，nmsl [/spoiler]，骂tgy是政治正确（非常确信），还有不知道为啥要位移的口水硬还是少来折腾点玩家了。\n        弱引导...虽然我觉得还凑合就是了，进了冰原不知道弄秘药吃的/如何使用材料快速调配的，找部分主线剧情脚印摸不到/进度不够的或者不看aibo提示找不到的，出门不吃猫饭（加下一任务内猫车回去前属性）的.....只能说还是得配合攻略玩吧（大概）\n       搓装备武器要的怨念物（稀有掉落物），在通关冰原后基本都能炼金炼出来，实在想要的可以去打有金框的调查任务，出货率要高很多，或者找人帮你一起刷，世界本体能凑体力增强三就好了(增加血量，一击线是最重要的)[spoiler] 实在不愿意就开老金魔出来，这点见仁见智，我因为冥赤武器词条花了五只的材料还刷不出来开过（知名粪怪），适当开有助于提高游戏体验[/spoiler]\n        友好或者一坨的社区（？）只能说小团体确实存在，但是基本上只要你肯发救援信